Notebook which will:
- Obtain alert photometry for named SN and create alert.
- Check whether this passes through a filter.

In [ ]:
import requests, os
from ampel.log.AmpelLogger import AmpelLogger
from ampel.contrib.hu.t0.SimpleDecentFilter import SimpleDecentFilter
from ampel.ztf.alert.ZiAlertSupplier import ZiAlertSupplier

In [ ]:
logger = AmpelLogger.get_logger()

In [ ]:
# Access to the AMPEL data archive assumes an individual *archive token* which can be obtained from 
# https://ampel.zeuthen.desy.de/live/dashboard/tokens
# This currently builds on GitHub authorization for AmpelProject members.
token = os.environ["ARCHIVE_TOKEN"]

In [ ]:
# Name of ZTF objects to lookup
snname = "ZTF22aafoqrd"

In [ ]:
def api_to_alert(name, token, tags=['FilterTest']):
    """
    Retrieve the alert history of a SN and return all recreated alerts.    
    """
    
    # Setup connection
    endpoint = "https://ampel.zeuthen.desy.de/api/ztf/archive/v3/object/{}/alerts?with_history=true".format(name)

    header = {"Authorization": "bearer "+token}

    response = requests.get(endpoint, headers=header )
    
    if not response.ok:
        print('... failed to get alert')
        return None
    
    return [ZiAlertSupplier.shape_alert_dict( a, tags ) for a in response.json()]

In [ ]:
alerts = api_to_alert(snname, token)

In [ ]:
print('Found %s alerts'%(len(alerts)))

Defining parameters for filter.

In [ ]:
filter_config = {
    'min_ndet': 1,
    'min_tspan': 0,
    'max_tspan': 50,
    'min_archive_tspan': 0,
    'max_archive_tspan': 500,    
    'min_rb': 0.3,
    'min_drb': 0.995,
    'max_fwhm': 5.5,
    'min_gal_lat': 14,
    'ps1_sgveto_rad': 1,
    'ps1_sgveto_th': 0.8,
    'ps1_confusion_rad': 3,
    'ps1_confusion_sg_tol': 0.1
}

In [ ]:
t0filter = SimpleDecentFilter( **filter_config, logger=logger )
t0filter.post_init()

In [ ]:
for alert in alerts:
    foo = t0filter.process(alert)
    print(foo)

In [ ]:
for a in alerts:
    latest = a.datapoints[0]
    # drb
    try:
        print('drb', latest['drb'])
    except KeyError:
        print('no drb')
    # rb
    try:
        print('rb', latest['rb'])
    except KeyError:
        print('no rb')
    # isdiffpos
    try:
        print('isdiffpos', latest['isdiffpos'])
    except KeyError:
        print('no isdiffpos')


In [ ]:
alerts[0].datapoints